In [1]:
#Get Neo4J official Python driver
!pip install neo4j

In [136]:
def connect(uri, username, password):
    try:
        return GraphDatabase.driver(uri, auth=(username, password))
    except ClientError as e:
        print(e)
        return -1

In [149]:
def create_node(tx, tup):
    url,subpages,metadata = tup[:3]
    #subpages=subpages[:5]
    tx.run("MERGE (main:Url {url: $url}) "
           "SET main = $properties, main.url = $url "
           "FOREACH (sub IN $subpages| "
                    "MERGE (u:Url {url: sub})"
                    "MERGE (main) -[:CONTAINS]-> (u)) ",
           url=url, subpages=subpages, properties=metadata)

In [138]:
def add_constraint(tx):
    tx.run("CREATE CONSTRAINT ON (u:Url) ASSERT u.url IS UNIQUE") 

In [139]:
def add_user(tx, username, password):
    try:
        result = tx.run("CALL dbms.security.createUser($username,$password, false)", username=username, password=password)
    except ClientError as e:
        print(e)

In [140]:
def change_password(tx, password):
    try:
        result = tx.run("CALL dbms.security.changePassword($password)", password=password)
    except ClientError as e:
        print(e)

In [141]:
def create_metadata(soup):
    title = soup.find("meta",  property="og:title")
    desc = soup.find("meta",  property="og:description")
    language = soup.find("meta",  property="og:locale")
    keywords = soup.find("meta", attrs = {'name':"keywords"})
    metadata = {
        'Title': title['content'] if title else None,
        'Description': desc['content']if desc else None,
        'Language': language['content'] if language else None,
        'Keywords': keywords['content'] if keywords else None}
    return metadata

In [142]:
def base_not_in_visted(url, table):
    if url in table: 
        return False
    if url.find('?ref=') != -1:
        if url[0:url.find('?ref=')] in table:
            return False
    for v in table:
        if v.find('?ref=') != -1:
            if v[0:v.find('?ref=')] == url:
                return False
            if url.find('?ref=') != -1 and v[0:v.find('?ref=')] == url[0:url.find('?ref=')]:
                return False
    return True          

In [ ]:
#Get Requests
!pip3 install beautifulsoup4

In [ ]:
#Simple get example
import requests

r = requests.get('https://www.agh.edu.pl/')
r.status_code
r.headers['content-type']
r.encoding
# r.text holds html file

In [162]:
#TODO : cut everything after ? -> IMPORTANT

#Crawler workflow
import requests, re
from bs4 import BeautifulSoup         
    
queue_limit = 100
visited = set()
#visited_base = set()
to_visit = ['http://home.agh.edu.pl/~ernst']
crawled_host = 'http://home.agh.edu.pl/'

results = []

while len(to_visit) > 0 and len(results) < queue_limit :

    crawled_url = to_visit.pop()
    urls = set()

    if crawled_url not in visited:
        r = requests.get(crawled_url)
        visited.add(crawled_url)

        if r.status_code == 200:
            soup = BeautifulSoup(r.text)
            
            for a in soup.find_all('a', href=True):
                current_path = a['href']
            
                if current_path.startswith('/') and not '#' in current_path:
                    current_url = crawled_host + current_path

                    if current_url.find('?ref=') == -1 and not current_url.endswith('/'):
                        current_url += '/' 

                    if base_not_in_visted(current_url, visited) and base_not_in_visted(current_url, to_visit): # not in visited:
                        urls.add(current_url)
                        to_visit.append(current_url)
           
            metadata = create_metadata(soup)
            result = (crawled_url, list(urls), metadata)
            results.append(result)
        else:
            print(r.status_code)


In [161]:
results

[('http://home.agh.edu.pl/~ernst',
  ['http://home.agh.edu.pl/~ernst/~ernst/dydaktyka:ztb:start/',
   'http://home.agh.edu.pl/~ernst/~ernst/dydaktyka:itik:start/',
   'http://home.agh.edu.pl/~ernst/~ernst/start?do=media&ns=/',
   'http://home.agh.edu.pl/~ernst/~ernst/dydaktyka:bdiszb:start/',
   'http://home.agh.edu.pl/~ernst/~ernst/start?do=recent/',
   'http://home.agh.edu.pl/~ernst/~ernst/start?do=revisions/',
   'http://home.agh.edu.pl/~ernst/~ernst/dydaktyka:baw:edycja7/',
   'http://home.agh.edu.pl/~ernst/~ernst/dydaktyka:wtm:start/',
   'http://home.agh.edu.pl/~ernst/~ernst/start?do=edit/',
   'http://home.agh.edu.pl/~ernst/~ernst/start?do=login&sectok=b6c373c8d3a2b90accbc3ba0d0c912e6/',
   'http://home.agh.edu.pl/~ernst/~ernst/dydaktyka:vlo_mobile:start/',
   'http://home.agh.edu.pl/~ernst/~ernst/dydaktyka:jitzw:start/',
   'http://home.agh.edu.pl/~ernst/~ernst/dla_studentow/',
   'http://home.agh.edu.pl/~ernst/~ernst/dydaktyka:ams:start/',
   'http://home.agh.edu.pl/~ernst/~er

In [146]:
#Neo4J connection test (Neo4j must be running on your localhost)
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError

uri = "bolt://localhost:7687"
our_username = "ads"
our_password = "ads"
message = "Response"
default = "neo4j"

# Logging with default user and password
driver = connect(uri, default, default)
# If we have already run this before, then the password is already changed
if driver != -1:
    ''' But if it is the first time we log as that default user, 
        we are forced to change the password '''
    with driver.session() as session:
        session.write_transaction(change_password, our_password) # e.g. our_password
    # We should exit and connect again after changing the password
    driver.close()

driver = connect(uri, default, our_password)

# Add the user that will be used for the rest of operations
with driver.session() as session:
    try: 
        session.write_transaction(add_user, our_username, our_password)
    except ClientError as e:
        print(e)
driver.close()


The client is unauthorized due to authentication failure.
The specified user 'ads' already exists.


In [147]:
driver = connect(uri, our_username, our_password)
with driver.session() as session:
    session.write_transaction(add_constraint)
driver.close()

In [163]:
driver = connect(uri, our_username, our_password)
with driver.session() as session:
    for res in results:
        session.write_transaction(create_node, res)
driver.close()

In [14]:
'''
driver = connect(uri, our_username, our_password)
with driver.session() as session:
    greeting = session.write_transaction(create_and_return_greeting, message)
    print(greeting) 
driver.close() '''

'''cqlCreate = "CREATE (neo4j:crawled_url{name:{}})".format(results[0][0])
driver = connect(uri, our_username, our_password)
with driver.session() as session:
    session.run(cqlCreate)'''

'cqlCreate = "CREATE (neo4j:crawled_url{name:{}})".format(results[0][0])\ndriver = connect(uri, our_username, our_password)\nwith driver.session() as session:\n    session.run(cqlCreate)'